In [1]:
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor, SAM2Base

predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-large")

In [2]:
# Prepare SAM 2
model = predictor.model.cuda()

image_encoder = model.image_encoder
memory_encoder = model.memory_encoder
memory_attention = model.memory_attention
image_decoder = model.sam_mask_decoder

In [3]:
# Examaple: Use 3 Images as Memory
from PIL import Image
memory_image_1 = Image.open('/home/kdh/code/BME_Capstone1/FAISS/A00028185_axial_107.png')
memory_label_1 = Image.open('/home/kdh/code/BME_Capstone1/FAISS/A00028185_axial_107_mask.png')

memory_image_2 = Image.open('/home/kdh/code/BME_Capstone1/FAISS/A00028185_axial_108.png')
memory_label_2 = Image.open('/home/kdh/code/BME_Capstone1/FAISS/A00028185_axial_108_mask.png')

memory_image_3 = Image.open('/home/kdh/code/BME_Capstone1/FAISS/A00028185_axial_109.png')
memory_label_3 = Image.open('/home/kdh/code/BME_Capstone1/FAISS/A00028185_axial_109_mask.png')

In [4]:
from preprocessing import preprocessing_sam2

In [5]:
mem1 = preprocessing_sam2(memory_image_1).cuda()
mem1.shape

torch.Size([3, 256, 192])

In [14]:
imgfeat = image_encoder(mem1.unsqueeze(0))

In [39]:
image_encoder.forward

<bound method ImageEncoder.forward of ImageEncoder(
  (trunk): Hiera(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 144, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
    )
    (blocks): ModuleList(
      (0-1): 2 x MultiScaleBlock(
        (norm1): LayerNorm((144,), eps=1e-06, elementwise_affine=True)
        (attn): MultiScaleAttention(
          (qkv): Linear(in_features=144, out_features=432, bias=True)
          (proj): Linear(in_features=144, out_features=144, bias=True)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((144,), eps=1e-06, elementwise_affine=True)
        (mlp): MLP(
          (layers): ModuleList(
            (0): Linear(in_features=144, out_features=576, bias=True)
            (1): Linear(in_features=576, out_features=144, bias=True)
          )
          (act): GELU(approximate='none')
        )
      )
      (2): MultiScaleBlock(
        (norm1): LayerNorm((144,), eps=1e-06, elementwise_affine=True)
        (pool): MaxPool2d(ke

In [35]:
imgfeat.keys()

dict_keys(['vision_features', 'vision_pos_enc', 'backbone_fpn'])

In [16]:
imgfeat['vision_features'].shape

torch.Size([1, 256, 16, 12])

In [6]:
mem1feat = image_encoder(mem1.unsqueeze(0))
mem1feat.keys()

dict_keys(['vision_features', 'vision_pos_enc', 'backbone_fpn'])

In [29]:
import numpy as np
mem1_mask = np.array(memory_label_1)
mask_for_mem = torch.Tensor(np.where(mem1_mask>0, 1, 0)).unsqueeze(0).cuda()

pix_feat = mem1feat

In [31]:
membank = memory_encoder(pix_feat['vision_features'], mask_for_mem, skip_mask_sigmoid=True)

In [34]:
membank#.keys()

{'vision_features': tensor([[[[-1.4846e+00, -2.4006e+00, -2.0437e+00,  ..., -1.5841e-03,
            -1.5754e+00, -2.3761e+00],
           [-3.4426e-01, -3.2450e-03, -7.3652e-01,  ..., -4.0971e-01,
             1.4022e-01, -6.6891e-03],
           [-3.5967e+00, -3.7593e-01, -7.0714e-01,  ...,  7.6252e-01,
             2.4212e-01, -1.2130e+00],
           ...,
           [-4.6076e-01,  5.3511e-02, -1.6299e+00,  ...,  9.2657e-02,
             2.5566e-01,  2.7983e-01],
           [-5.8862e-01,  4.1013e-02, -9.6927e-01,  ...,  8.9323e-02,
            -6.0268e-01, -1.4948e+00],
           [-8.9012e-01, -1.2158e+00, -9.3918e-01,  ...,  3.5316e-01,
            -1.1954e+00, -1.5261e+00]],
 
          [[-5.8099e-01, -9.7041e-01, -3.2968e-01,  ...,  1.0022e-01,
             6.5936e-02, -5.1423e-01],
           [ 1.5392e-01, -7.1067e-02, -9.9963e-01,  ..., -1.2612e+00,
             6.6722e-02,  6.1804e-01],
           [ 4.4484e-01, -7.6338e-02, -1.5921e+00,  ..., -1.0166e+00,
             6.4852e